In [1]:
from typing import Tuple
import pandas as pd
from scipy.stats import pearsonr
from tqdm.notebook import tqdm
import sys
sys.path.append('..')
from evaluation.generated_dataset import GeneratedDataset, load_all_from_config
from evaluation.novelty import NoveltyFilter, filter_by_unique_structure_reduced_comp_index, record_to_anonymous_fingerprint

/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/matminer/utils/data.py:326: UserWarning: MagpieData(impute_nan=False):
In a future release, impute_nan will be set to True by default.
                    This means that features that are missing or are NaNs for elements
                    from the data source will be replaced by the average of that value
                    over the available elements.
                    This avoids NaNs after featurization that are often replaced by
                    dataset-dependent averages.
  warnings.warn(f"{self.__class__.__name__}(impute_nan=False):\n" + IMPUTE_NAN_WARNING)


In [2]:
dft_datasets = {
    # These DFT used wrong pseudopotentials, and was not recomputed
    # "WyFormerDirect": ("WyckoffTransformer", "DFT"),
    "WyFormerCrySPR": ("WyckoffTransformer", "CrySPR", "CHGNet_fix", "DFT"),
    "WyFormerDiffCSP++": ("WyckoffTransformer", "DiffCSP++", "DFT"),
    "WyFormerDiffCSP++10k": ("WyckoffTransformer", "DiffCSP++10k", "CHGNet_free", "DFT"),
    "WyFormerDiffCSP++10k-GGA-relax-1": ("WyckoffTransformer", "DiffCSP++10k", "CHGNet_free", "DFT-GGA-relax-1"),
    "WyFormerHarmonicDiffCSP++": ("WyckoffTransformer-harmonic", "DiffCSP++", "DFT"),
    "WyLLM-DiffCSP++": ("WyckoffLLM-naive", "DiffCSP++", "DFT"),
    "WyFormer-letters-DiffCSP++": ("WyckoffTransformer-letters", "DiffCSP++", "DFT"),
    "SymmCD": ("SymmCD", "DFT"),
    "DiffCSP": ("DiffCSP", "DFT"),
    "CrystalFormer": ("CrystalFormer", "DFT"),
    "DiffCSP++": ("DiffCSP++", "DFT"),
    "FlowMM": ("FlowMM", "DFT"),
    "MatterGen": ("MatterGen", "MatterGen_10k", "DFT"),
    "MatterGen-CHGNet": ("MatterGen", "MatterGen_10k", "CHGNet_fix", "DFT")
}

source_datasets = {name: t[:-1] for name, t in dft_datasets.items()}

In [3]:
chgnet_datasets = {
    "WyFormerDirect": ("WyckoffTransformer", "CrySPR", "CHGNet_fix_release"),
    "WyFormerCrySPR": ("WyckoffTransformer", "CrySPR", "CHGNet_fix_release"),
    "WyFormerDiffCSP++": ("WyckoffTransformer", "DiffCSP++", "CHGNet_fix"),
    "WyFormerDiffCSP++10k": ("WyckoffTransformer", "DiffCSP++10k", "CHGNet_free"),
    "WyFormerDiffCSP++10k-GGA-relax-1": ("WyckoffTransformer", "DiffCSP++10k", "CHGNet_free"),
    "WyLLM-DiffCSP++": ("WyckoffLLM-naive", "DiffCSP++", "CHGNet_fix"),
    "WyFormer-letters-DiffCSP++": ("WyckoffTransformer-letters", "DiffCSP++", "CHGNet_fix"),
    "WyFormerHarmonicDiffCSP++": ("WyckoffTransformer-harmonic", "DiffCSP++", "CHGNet_fix"),
    "SymmCD": ("SymmCD", "CHGNet_fix"),
    "DiffCSP": ("DiffCSP", "CHGNet_fix"),
    "CrystalFormer": ("CrystalFormer", "CHGNet_fix_release"),
    "DiffCSP++": ("DiffCSP++", "CHGNet_fix_release"),
    "FlowMM": ("FlowMM", "CHGNet_fix"),
    "MatterGen": ("MatterGen", "MatterGen_10k", "CHGNet_fix"),
    "MatterGen-CHGNet": ("MatterGen", "MatterGen_10k", "CHGNet_fix")
}

In [4]:
chgnet_data = load_all_from_config(datasets=list(chgnet_datasets.values()) + [('WyckoffTransformer', 'CrySPR', 'CHGNet_fix')])

In [5]:
all_datasets = load_all_from_config(
    datasets=list(dft_datasets.values()) + list(source_datasets.values()) + \
        [("split", "train"), ("split", "val"), ("split", "test")],
    dataset_name="mp_20")

In [6]:
wycryst_transformations = ('WyCryst', 'CrySPR', 'CHGNet_fix')
source_datasets["WyCryst"] = wycryst_transformations
chgnet_datasets["WyCryst"] = wycryst_transformations
chgnet_data[wycryst_transformations] = GeneratedDataset.from_cache(wycryst_transformations, "mp_20_biternary")
dft_datasets["WyCryst"] = tuple(list(wycryst_transformations) + ["DFT"])
all_datasets[dft_datasets["WyCryst"]] = GeneratedDataset.from_cache(dft_datasets["WyCryst"], "mp_20_biternary")

In [7]:
novelty_reference = pd.concat([
    all_datasets[('split', 'train')].data,
    all_datasets[('split', 'val')].data], axis=0, verify_integrity=True)
novelty_filter = NoveltyFilter(novelty_reference, reference_index_type="reduced_composition")
novelty_filter_fingerprint = NoveltyFilter(novelty_reference, reference_index_type="fingerprint")

/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/pymatgen/core/composition.py:1365: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms: list[str] = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])
/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/pymatgen/core/composition.py:1365: UserWarning: No Pauling electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms: list[str] = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])


In [8]:
import evaluation.statistical_evaluator
test_evaluator = evaluation.statistical_evaluator.StatisticalEvaluator(all_datasets[('split', 'test')].data)

In [9]:
train_w_template_set = frozenset(novelty_reference.apply(record_to_anonymous_fingerprint, axis=1))

In [10]:
def is_sg_preserved(relaxed_sg, transformations: Tuple[str]) -> pd.Series:
    source_sg = all_datasets[transformations[:-1]].data.spacegroup_number
    return relaxed_sg == source_sg.reindex_like(relaxed_sg)

In [11]:
train_novelty_filter = NoveltyFilter(all_datasets[('split', 'train')].data, reference_index_type="reduced_composition")

/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/pymatgen/core/composition.py:1365: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms: list[str] = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])
/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/pymatgen/core/composition.py:1365: UserWarning: No Pauling electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms: list[str] = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])


In [12]:
def get_train_based_sun(dataset: pd.DataFrame, intial_count: int):
    unique = filter_by_unique_structure_reduced_comp_index(dataset)
    print("Unique structures %", len(unique)/len(dataset) * 100)
    unique_novel = train_novelty_filter.get_novel(unique)
    print("Novel & Unique structures %", len(unique_novel)/len(dataset) * 100)
    for threshold in (0, 0.08):
        sun = ((unique_novel.e_above_hull_corrected < threshold) & unique_novel.structure.map(lambda s: len(set(s.composition)) >=2)).sum()
        print(f"S.U.N. (E<{threshold}) %: {sun/intial_count * 100:.2f})")

In [13]:
get_train_based_sun(all_datasets[dft_datasets["WyFormerDiffCSP++10k"]].data, 1e4)

Unique structures % 98.80494648238594


/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/pymatgen/core/composition.py:1365: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms: list[str] = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])


Novel & Unique structures % 91.14621219993765
S.U.N. (E<0) %: 3.78)
S.U.N. (E<0.08) %: 18.54)


In [14]:
get_train_based_sun(all_datasets[dft_datasets["WyFormerDiffCSP++10k-GGA-relax-1"]].data, 1e4)

Unique structures % 98.80855397148676


/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/pymatgen/core/composition.py:1365: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms: list[str] = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])


Novel & Unique structures % 91.17107942973523
S.U.N. (E<0) %: 4.09)
S.U.N. (E<0.08) %: 18.83)


Validity
1. Vanilla; Valid records: 2866 / 9648 = 29.71%
2. Naive; Valid records: 9492 / 9804 = 96.82%
3. Site Symmetry; Valid records: 8955 / 9709 = 92.23%

In [ ]:
tables = {}
for E_hull_threshold in (0, 0.08):
    table = pd.DataFrame(
        index=dft_datasets.keys(), columns=[
            "DFT dataset size",
            "Source Novelty (%)",
            "In-DFT Novelty (%)",
            "S.U.N. (%)",
            "P1 in source (%)",
            "S.S.U.N. (%)"])
    table.index.name = "Method"

    for name, transformations in tqdm(dft_datasets.items()):
        dataset = all_datasets[transformations]
        if "corrected_e_hull" not in dataset.data.columns:
            dataset.data["corrected_e_hull"] = dataset.data.e_above_hull_corrected
        table.loc[name, "DFT dataset size"] = len(dataset.data)
        try:
            source_dataset = all_datasets[transformations[:-1]]
        except KeyError:
            source_dataset = chgnet_data[transformations[:-1]]
        chgnet_dataset = chgnet_data[chgnet_datasets[name]]

        unique = filter_by_unique_structure_reduced_comp_index(dataset.data)
        novel = novelty_filter.get_novel(unique)
        table.loc[name, "In-DFT Novelty (%)"] = 100 * len(novel) / len(unique)
        if "structure" not in source_dataset.data.columns:
            print(f"Skipping {name} as the source does not have structure column")
            continue
        # Fingerprint novlety was used for sampling, and is a very slight overestimation
        # of the reduced composition novelty. But this is not a problem. If a structure is mistakenly
        # considered novel, it is just sent for DFT and subsequent roubst filtering.
        source_novel = novelty_filter_fingerprint.get_novel(source_dataset.data)
        source_novelty = 100 * len(source_novel) / len(source_dataset.data)
        table.loc[name, "Source Novelty (%)"] = len(novel) / len(unique) * source_novelty
        table.loc[name, "P1 in source (%)"] = 100 * (source_novel.group == 1).mean()
        try:
            table.loc[name, "SG preserved (%)"] = 100 * is_sg_preserved(novel.spacegroup_number, transformations).mean()
        except KeyError:
            pass
        # source_novel_symmetric = (source_novel.group != 1).sum() / len(source_dataset.data)
        # table["Source Novel !P1 (%)"] = 100 * source_novel_symmetric
        # DFT failure == unreal structure
        if name == "WyFormerDiffCSP++10k":
            # This dataset is a bit special, as it contains 10k structures from the WyckoffTransformer
            # not filtered by the novelty filter
            dft_structures = 10000
            source_novelty = 100
        else:
            dft_structures = 105
        has_ehull = dataset.data.corrected_e_hull.notna()
        is_sun = (novel.corrected_e_hull <= E_hull_threshold) # & (novel.elements.apply(lambda x: len(frozenset(x))) >= 2)
        table.loc[name, "S.U.N. (%)"] = source_novelty * is_sun.sum() / dft_structures
        table.loc[name, "total_sun"] = is_sun.sum().astype(int)
        table.loc[name, "S.S.U.N. (%)"] = source_novelty * (is_sun & (novel.group != 1)).sum() / dft_structures
        table.loc[name, "total_ssun"] = (is_sun & (novel.group != 1)).sum().astype(int)
        table.loc[name, "P1 in stable (%)"] = 100 * (novel[is_sun].group == 1).mean()

        chgnet_unique = filter_by_unique_structure_reduced_comp_index(chgnet_dataset.data)
        chgnet_novel = novelty_filter.get_novel(chgnet_unique)
        if "corrected_chgnet_ehull" in chgnet_dataset.data.columns:
            chgnet_is_sun = (chgnet_novel.corrected_chgnet_ehull < E_hull_threshold)
            #table.loc[name, "CHGNet dataset size"] = chgnet_dataset.data.corrected_chgnet_ehull.notna().sum()
            table.loc[name, "S.U.N. (CHGNet) (%)"] =  100 * chgnet_is_sun.sum() / chgnet_dataset.data.corrected_chgnet_ehull.notna().sum()
            table.loc[name, "S.S.U.N. (CHGNet) (%)"] = 100 * (chgnet_is_sun & (chgnet_novel.group != 1)).sum() / chgnet_dataset.data.corrected_chgnet_ehull.notna().sum()
            
            chgnet_dft_available = chgnet_dataset.data.reindex(dataset.data.index[has_ehull])
            table.loc[name, "r DFT CHGNet"] = \
                pearsonr((chgnet_dft_available.corrected_chgnet_ehull < E_hull_threshold).astype(float),
                        (dataset.data.corrected_e_hull < E_hull_threshold).astype(float)).correlation
    tables[E_hull_threshold] = table

  0%|          | 0/15 [00:00<?, ?it/s]

/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/pymatgen/core/composition.py:1365: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms: list[str] = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])
/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/pymatgen/core/composition.py:1365: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms: list[str] = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])
/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/pymatgen/core/composition.py:1365: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly

  0%|          | 0/15 [00:00<?, ?it/s]

/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/pymatgen/core/composition.py:1365: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms: list[str] = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])
/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/pymatgen/core/composition.py:1365: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms: list[str] = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])
/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/pymatgen/core/composition.py:1365: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly

In [16]:
tables[0]

,DFT dataset size,Source Novelty (%),In-DFT Novelty (%),S.U.N. (%),P1 in source (%),S.S.U.N. (%),SG preserved (%),total_sun,total_ssun,P1 in stable (%),S.U.N. (CHGNet) (%),S.S.U.N. (CHGNet) (%),r DFT CHGNet
Method,,,,,,,,,,,,,
WyFormerCrySPR,104,90.990991,100.0,4.332904,1.540154,4.332904,96.153846,5.0,5.0,0.000000,13.713714,13.613614,0.323818
WyFormerDiffCSP++,104,89.629808,99.038462,4.309524,1.546961,4.309524,95.145631,5.0,5.0,0.000000,12.400000,12.400000,0.334818
WyFormerDiffCSP++10k,9623,82.129635,90.145141,3.45,7.004062,3.4,92.276280,345.0,340.0,1.449275,NaN,NaN,NaN
WyFormerDiffCSP++10k-GGA-relax-1,9820,82.169231,90.188601,321.048019,7.004062,314.974138,93.863558,370.0,363.0,1.891892,NaN,NaN,NaN
WyFormerHarmonicDiffCSP++,101,89.786139,98.019802,4.361905,2.401747,4.361905,91.919192,5.0,5.0,0.000000,7.731434,7.629705,0.359760
WyLLM-DiffCSP++,102,94.063672,99.019608,4.523571,1.369863,4.523571,99.009901,5.0,5.0,0.000000,7.441386,7.339450,0.778248
WyFormer-letters-DiffCSP++,104,87.267339,97.115385,3.423217,1.117318,3.423217,96.039604,4.0,4.0,0.000000,8.342023,8.342023,0.529150
SymmCD,96,86.507916,95.833333,4.29853,2.186367,4.29853,92.391304,5.0,5.0,0.000000,9.728601,9.707724,-0.097590
DiffCSP,104,89.289231,98.076923,6.936381,31.140158,6.069333,78.431373,8.0,7.0,12.500000,17.400000,14.700000,0.441401


In [17]:
tables[0.08]

,DFT dataset size,Source Novelty (%),In-DFT Novelty (%),S.U.N. (%),P1 in source (%),S.S.U.N. (%),SG preserved (%),total_sun,total_ssun,P1 in stable (%),S.U.N. (CHGNet) (%),S.S.U.N. (CHGNet) (%),r DFT CHGNet
Method,,,,,,,,,,,,,
WyFormerCrySPR,104,90.990991,100.0,23.397683,1.540154,22.531103,96.153846,27.0,26.0,3.703704,39.139139,38.138138,0.664923
WyFormerDiffCSP++,104,89.629808,99.038462,22.409524,1.546961,21.547619,95.145631,26.0,25.0,3.846154,36.600000,35.900000,0.615387
WyFormerDiffCSP++10k,9623,82.129635,90.145141,16.7,7.004062,16.29,92.276280,1670.0,1629.0,2.455090,NaN,NaN,NaN
WyFormerDiffCSP++10k-GGA-relax-1,9820,82.169231,90.188601,1471.614704,7.004062,1434.303718,93.863558,1696.0,1653.0,2.535377,NaN,NaN,NaN
WyFormerHarmonicDiffCSP++,101,89.786139,98.019802,18.32,2.401747,18.32,91.919192,21.0,21.0,0.000000,33.570702,32.553408,0.593771
WyLLM-DiffCSP++,102,94.063672,99.019608,10.856571,1.369863,10.856571,99.009901,12.0,12.0,0.000000,30.886850,30.173293,0.592083
WyFormer-letters-DiffCSP++,104,87.267339,97.115385,16.260279,1.117318,16.260279,96.039604,19.0,19.0,0.000000,29.197080,28.675704,0.548767
SymmCD,96,86.507916,95.833333,17.19412,2.186367,17.19412,92.391304,20.0,20.0,0.000000,32.640919,32.160752,0.108845
DiffCSP,104,89.289231,98.076923,22.543238,31.140158,20.809143,78.431373,26.0,24.0,7.692308,56.700000,39.900000,0.405781


In [18]:
all_datasets[('split', 'test')].data.apply(evaluation.novelty.record_to_anonymous_fingerprint, axis=1).isin(train_w_template_set).mean()

0.9713685606898077

In [19]:
from scipy.stats import ttest_ind
import numpy as np
table = tables[0.08]
def get_observation(name, column="total_ssun"):
    all_observations = np.zeros(dft_structures)
    all_observations[:int(table.at[name, column])] = table.loc[name, "Source Novelty (%)"]/100
    return all_observations

In [20]:
for second in table.index:
    print(second, ttest_ind(get_observation("WyFormerCrySPR"), get_observation(second)))

WyFormerCrySPR TtestResult(statistic=0.0, pvalue=1.0, df=208.0)
WyFormerDiffCSP++ TtestResult(statistic=0.22169842125548883, pvalue=0.8247661656005018, df=208.0)
WyFormerDiffCSP++10k TtestResult(statistic=-15.475436894287114, pvalue=1.877425531979627e-36, df=208.0)
WyFormerDiffCSP++10k-GGA-relax-1 TtestResult(statistic=-15.485718476135018, pvalue=1.7430600257132496e-36, df=208.0)
WyFormerHarmonicDiffCSP++ TtestResult(statistic=0.8764545695078809, pvalue=0.3817940550261041, df=208.0)
WyLLM-DiffCSP++ TtestResult(statistic=2.4331627543031393, pvalue=0.015812465316830185, df=208.0)
WyFormer-letters-DiffCSP++ TtestResult(statistic=1.3298945441547065, pvalue=0.18500950699035804, df=208.0)
SymmCD TtestResult(statistic=1.1888391591707268, pvalue=0.2358584457291209, df=208.0)
DiffCSP TtestResult(statistic=0.39857264960251715, pvalue=0.6906169643892961, df=208.0)
CrystalFormer TtestResult(statistic=0.604255584143725, pvalue=0.5463322450046239, df=208.0)
DiffCSP++ TtestResult(statistic=1.79711373

/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:579: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)


In [21]:
for second in table.index:
    print(second, ttest_ind(get_observation("WyFormerCrySPR", column="total_sun"), get_observation(second, column="total_sun")))

WyFormerCrySPR TtestResult(statistic=0.0, pvalue=1.0, df=208.0)
WyFormerDiffCSP++ TtestResult(statistic=0.22123986882655103, pvalue=0.8251227040415868, df=208.0)
WyFormerDiffCSP++10k TtestResult(statistic=-15.06096575659138, pvalue=3.757490418936178e-35, df=208.0)
WyFormerDiffCSP++10k-GGA-relax-1 TtestResult(statistic=-15.071119611967125, pvalue=3.4913345891329897e-35, df=208.0)
WyFormerHarmonicDiffCSP++ TtestResult(statistic=1.0353972199054013, pvalue=0.30168569960769603, df=208.0)
WyLLM-DiffCSP++ TtestResult(statistic=2.5914698498126034, pvalue=0.010233727725327985, df=208.0)
WyFormer-letters-DiffCSP++ TtestResult(statistic=1.490034214181806, pvalue=0.13773004700560512, df=208.0)
SymmCD TtestResult(statistic=1.3492933036822556, pvalue=0.17870965805453398, df=208.0)
DiffCSP TtestResult(statistic=0.237178917078767, pvalue=0.8127514297668421, df=208.0)
CrystalFormer TtestResult(statistic=0.7690063680373264, pvalue=0.4427618523055933, df=208.0)
DiffCSP++ TtestResult(statistic=1.956608663

In [22]:
table

,DFT dataset size,Source Novelty (%),In-DFT Novelty (%),S.U.N. (%),P1 in source (%),S.S.U.N. (%),SG preserved (%),total_sun,total_ssun,P1 in stable (%),S.U.N. (CHGNet) (%),S.S.U.N. (CHGNet) (%),r DFT CHGNet
Method,,,,,,,,,,,,,
WyFormerCrySPR,104,90.990991,100.0,23.397683,1.540154,22.531103,96.153846,27.0,26.0,3.703704,39.139139,38.138138,0.664923
WyFormerDiffCSP++,104,89.629808,99.038462,22.409524,1.546961,21.547619,95.145631,26.0,25.0,3.846154,36.600000,35.900000,0.615387
WyFormerDiffCSP++10k,9623,82.129635,90.145141,16.7,7.004062,16.29,92.276280,1670.0,1629.0,2.455090,NaN,NaN,NaN
WyFormerDiffCSP++10k-GGA-relax-1,9820,82.169231,90.188601,1471.614704,7.004062,1434.303718,93.863558,1696.0,1653.0,2.535377,NaN,NaN,NaN
WyFormerHarmonicDiffCSP++,101,89.786139,98.019802,18.32,2.401747,18.32,91.919192,21.0,21.0,0.000000,33.570702,32.553408,0.593771
WyLLM-DiffCSP++,102,94.063672,99.019608,10.856571,1.369863,10.856571,99.009901,12.0,12.0,0.000000,30.886850,30.173293,0.592083
WyFormer-letters-DiffCSP++,104,87.267339,97.115385,16.260279,1.117318,16.260279,96.039604,19.0,19.0,0.000000,29.197080,28.675704,0.548767
SymmCD,96,86.507916,95.833333,17.19412,2.186367,17.19412,92.391304,20.0,20.0,0.000000,32.640919,32.160752,0.108845
DiffCSP,104,89.289231,98.076923,22.543238,31.140158,20.809143,78.431373,26.0,24.0,7.692308,56.700000,39.900000,0.405781


In [23]:
ttest_ind(
    (all_datasets[('WyckoffTransformer', 'CrySPR', 'CHGNet_fix')].data.group == 1).astype(float),
    (chgnet_data[('WyckoffTransformer', "DiffCSP++", "CHGNet_fix")].data.group == 1).astype(float))

TtestResult(statistic=0.19628385802590687, pvalue=0.8444079665608555, df=1997.0)

In [24]:

for second in table.index:
    print(second, ttest_ind(get_observation("WyFormerCrySPR", column="total_sun"), get_observation(second, column="total_sun")))

WyFormerCrySPR TtestResult(statistic=0.0, pvalue=1.0, df=208.0)
WyFormerDiffCSP++ TtestResult(statistic=0.22123986882655103, pvalue=0.8251227040415868, df=208.0)
WyFormerDiffCSP++10k TtestResult(statistic=-15.06096575659138, pvalue=3.757490418936178e-35, df=208.0)
WyFormerDiffCSP++10k-GGA-relax-1 TtestResult(statistic=-15.071119611967125, pvalue=3.4913345891329897e-35, df=208.0)
WyFormerHarmonicDiffCSP++ TtestResult(statistic=1.0353972199054013, pvalue=0.30168569960769603, df=208.0)
WyLLM-DiffCSP++ TtestResult(statistic=2.5914698498126034, pvalue=0.010233727725327985, df=208.0)
WyFormer-letters-DiffCSP++ TtestResult(statistic=1.490034214181806, pvalue=0.13773004700560512, df=208.0)
SymmCD TtestResult(statistic=1.3492933036822556, pvalue=0.17870965805453398, df=208.0)
DiffCSP TtestResult(statistic=0.237178917078767, pvalue=0.8127514297668421, df=208.0)
CrystalFormer TtestResult(statistic=0.7690063680373264, pvalue=0.4427618523055933, df=208.0)
DiffCSP++ TtestResult(statistic=1.956608663

/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-FeCwefly-py3.12/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:579: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  res = hypotest_fun_out(*samples, **kwds)
